In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torchvision
import torch.nn as nn

import matplotlib.pyplot as plt

In [121]:
# Generate training data. We use a compound lookup table, then add some noise.
data = pd.read_csv("train_data.csv")

# Select a bunch of random rows in the data
np.random.seed(2)
masses = []
species = []
for i in range(100):
    rand_index = np.random.randint(10)
    masses.append(data.at[rand_index, "Precise Mass"])
    species.append(data.at[rand_index, "Species"])
masses = np.array(masses)

# Generate noise for the same number of rows'
noise = 0.01 * np.random.standard_normal(size = 100)

# Combine the noise and randomly selected masses to create our training data
x_train = masses + noise
y_train = species
print("x_train: ", x_train)
print("y_train: ", y_train)

x_train:  [25.02673425 25.04207698 18.98346009 12.02303403 25.0341716  24.02093281
 12.01476112  2.0172077  17.00094655 16.00578802 16.00135855 17.01666921
 24.02365827 13.03376307 18.99872681 16.00354095 12.99321443 24.01442844
 18.98913247  2.01823093 13.00657972 17.00125768 25.03548677 16.00465218
 19.00741689 13.02029975 26.02199951 12.01393662  1.00933354 16.01135847
 12.00161484 15.98267865  2.01856522 24.0206913  25.04168372 12.00979752
 26.02745525 25.03040884 24.02355504  2.03085061 18.99404805 25.03650631
 17.00717863 26.00528869 26.00741805 26.01232931 13.02457721  1.00334771
  1.00757214 12.0112102  25.01543983 25.03839779 12.01296505 26.03039862
 18.99639306 17.01338926 18.99552835 18.99085791 18.99762262 13.01536759
 25.03052282 12.01109264  1.99623511 15.98973636 25.02956752  2.02679665
 19.00654154 26.02519478 17.01866149  2.00726363 12.00825392 15.99665323
 24.02983943 18.98681599 16.0063513  17.0103751  25.04028688 13.01003242
  0.99542513  1.00708217 17.00855242 24.0

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# TODO
# Normalize data before we pass it through our network.
scaler = StandardScaler()
data = scaler.fit_transform(mass_data.reshape(-1,1))

In [ ]:
#Define our model.
class speciesClassification(torch.nn.Module):
    
    def __init__(self, input_dim, output_dim): # Feel free to add hidden_dim as parameters here
        
        # TODO
        super(speciesClassification, self).__init__()
        
        # YOUR CODE HERE
        # Create input layer, 2 hidden layers of 32 neurons each with ReLu activation functions after each layer, and
        # output layer.
        self.layer1 = nn.Linear(input_dim, 32)
        self.actfun1 = nn.ReLU()
        self.layer2 = nn.Linear(32, 32)
        self.actfun2 = nn.ReLU()
        self.output_layer = nn.Linear(32, output_dim)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        
        # YOUR CODE HERE
        # Send our inputs through the network defined above.
        x = self.layer1(x)
        x = self.actfun1(x)
        x = self.layer2(x)
        x = self.actfun2(x)
        x = self.output_layer(x)
        out = self.softmax(x)
        
        return out

In [ ]:
# Initialize our neural network model with input and output dimensions. Inputs are images with 28 x 28 pixels, output is 
# a digit from 0-9.
model = speciesClassification(28**2, 10)

# Define the learning rate and epoch 
learning_rate = 0.01
epochs = 1000
batchsize = 50

# Define loss function and optimizer. Start out using SGD (TODO: CHANGE LATER IF NEEDED).
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# Run this line if you have PyTorch GPU version
if torch.cuda.is_available():
    model.cuda()

model

In [ ]:
# Placeholders for training loss and validation accuracy during training
# Training loss should be tracked for each iteration (1 iteration -> single forward pass to the network)
# Validation accuracy should be evaluated every 'Epoch' (1 epoch -> full training dataset)
# If using batch gradient, 1 iteration = 1 epoch

train_loss_list = []
validation_accuracy_list = []

In [ ]:
import tqdm

# Convert the training, validation, testing dataset (NumPy arrays) into torch tensors

# YOUR CODE HERE
x_train_final = torch.from_numpy(x_train).float()
y_train_final = torch.from_numpy(y_train).long()

validation_inputs = torch.from_numpy(x_validation).float()
validation_targets = torch.from_numpy(y_validation).long()

testing_inputs = torch.from_numpy(x_test).float()
testing_targets = torch.from_numpy(y_test).long()

# Training Loop ---------------------------------------------------------------------------------------

for epoch in tqdm.trange(epochs):
    
    # YOUR CODE HERE
    # Step through the optimizer and update the loss on each step.
    optimizer.zero_grad()
    outputs = model(x_train_final)

    # print(outputs.shape)
    # print(y_train_final.shape)
    
    loss = loss_func(outputs, y_train_final)
    train_loss_list.append(loss.item())
    loss.backward()
    optimizer.step()
    
    # Compute Validation Accuracy ----------------------------------------------------------------------
    
    # YOUR CODE HERE
    with torch.no_grad(): # Tell PyTorch that we aren't passing inputs to network for training purpose
        
        # Pass the validation feature data (30 samples) to the network
        validation_outputs = model(validation_inputs)
        
        # validation_outputs = tensor of shape (30, 3), i.e., 30 predictions and each prediction has 3 probabilities.
        # These are classification probabilities for each flower type: [p(setosa), p(versicolor), p(virginica)].
        # torch.argmax(validation_outputs, dim=1) finds the index with the maximum value alongside the column direction
        # i.e. for each sample, it finds the column index with the highest probability.
        # == validation_targets compares these indices with groundtruth validation target labels for each sample
        # For each sample, it returns True if the index matches the target, False otherwise
        # .type(torch.FloatTensor) converts True = 1, False = 0
        # Finally .mean() gives us (Total count of 1) / (Length of the 1D tensor), giving us the classification accuracy  
        
        correct = (torch.argmax(validation_outputs, dim=1) == 
                   validation_targets).type(torch.FloatTensor)
        
        validation_accuracy_list.append(correct.mean())

In [ ]:
# Import seaborn for prettier plots

import seaborn as sns
# Visualize training loss

plt.figure(figsize = (12, 6))

# Visualize training loss with respect to iterations (1 iteration -> single batch)
plt.subplot(2, 1, 1)
plt.plot(train_loss_list, linewidth = 3)
plt.ylabel("training loss")
plt.xlabel("epochs")
sns.despine()

# Visualize validation accuracy with respect to epochs
plt.subplot(2, 1, 2)
plt.plot(validation_accuracy_list, linewidth = 3, color = 'gold')
plt.ylabel("validation accuracy")
sns.despine()

In [ ]:
# YOUR CODE HERE
# Compute the testing accuracy.
with torch.no_grad(): # Tell PyTorch that we aren't passing inputs to network for training purpose.

    # Pass the testing feature data to the network to produce model predictions.
    y_pred_test = model(testing_inputs) 
    
    # Use the same technique as above to compute the testing classification accuracy.
    correct = (torch.argmax(y_pred_test, dim=1) == testing_targets).type(torch.FloatTensor)
    
    # Print test accuracy to the user.
    print("Testing Accuracy: " + str(correct.mean().numpy()*100) + '%')

In [ ]:
# Check dimensions of our prediction and actual arrays and print some useful things.
print(torch.argmax(y_pred_test, dim=1))
print(testing_inputs.numpy().shape)
print(y_pred_test.numpy().shape)
print(correct.numpy().shape)